In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen,Request

In [2]:
link_final = [] #finding https
review_final = [] #Review titles only
rating_final = [] #As a percentage
date_final = [] #month + date
months_final = [] 
years_final = []
albumlink_final = []
album_final = []
author_final = []
band_final = []
#texts_final = []#actual review texts
#sentiments_final = []

In [3]:
def scrape(year, month, phase, web = False):
    global link_final,review_final,rating_final,date_final,months_final,years_final,albumlink_final,album_final,author_final,band_final
    # Create a dict from the method arguments
    #sentiments = []
    link = [] #finding https
    review = [] #Review titles only
    rating = [] #As a percentage
    date = [] #month + date
    months = [] 
    years = []
    album = []
    author = []
    band = []
    albumlink = []
    #texts = []#actual review texts
    #analyzer = SentimentIntensityAnalyzer()
    tmpstr = str(month)
    if len(tmpstr) == 1:
        tmpstr = '0' + tmpstr
    if web == True:
        url = 'https://www.metal-archives.com/review/ajax-list-browse/by/date/selection/' + str(year) + '-' + tmpstr + '/'+ '?iDisplayStart='+ str(phase*200)
        req = Request(url=url, headers = {'User-Agent': 'Mozilla/5.0'}) 
        r = urlopen(req).read().decode()
    else:
        f = open('reviews_links/'+str(year)+'/'+tmpstr+'/'+str(year)+'-'+tmpstr+'_'+str(phase)+'.txt', 'r', encoding = 'utf-8')
        r = f.read()
        #print('reviews_links/'+str(year)+'/'+tmpstr+'/'+str(year)+'-'+tmpstr+'_'+str(phase)+'.txt')
    rawData = str(r).split('"')#converting json into string
    N = -1
    if rawData[2][2:6].isnumeric():
        N = int(rawData[2][2:6])
    elif rawData[2][2:5].isnumeric():
        N = int(rawData[2][2:5])
    elif rawData[2][2:4].isnumeric():
        N = int(rawData[2][2:4])
    elif year == 2002 and month < 7:
        pass
    else:
        print("Warning: N not found at position rawData[2].",year,month)
    month_list = ['January','February','March','April','May','June','July','August','September','October','November','December']#to check for date
    cnt = 1 #debugging counter
    prev = True #if previous word is part of the review
    for i in range(0,len(rawData)):
        word = rawData[i]
        if word[:5] == "https" and word.split('/')[3] == 'reviews':
            #check for the first link where information on band, album and author can be extracted.
            #This is also the link of the review which can be analysed further later
            link.append(word)
            tmp = word.split('/')
            band.append(tmp[4])
            album.append(tmp[5])
            author.append(tmp[7])
            '''r = requests.get(word)
            html_doc = r.content
            soup = BeautifulSoup(html_doc, 'html.parser')
            raw_text = soup.find_all('p')[3].text.replace('\n','')
            texts.append(raw_text)
            sentiments.append(analyzer.polarity_scores(raw_text)['compound'])'''
            #print(cnt,word)
            #cnt += 1
        elif word[:5] == "https" and word.split('/')[3] == 'albums':
            albumlink.append(word)
        elif (rawData[i-1][:7] == ' title=') and word[:7] != ' class=':
            #find the title of the review
            review.append(word[:-1])
            i1 = i + 1
            while i1<len(rawData) and rawData[i1][:7] != ' class=':#when reviews have ""
                #print(i1)
                review[-1] += rawData[i1][:-1]
                i1 += 1
                #print(review[-1])
            #print(cnt,rawData[i+1][0] == ' ')

            #print(cnt,rawData[i-1])
            #cnt += 1
            
        elif len(word) > 0 and word.split()[0] == month_list[month-1]:#eg. July 1
            #find the date
            date.append(word + ' ' + str(year))
            months.append(month)
            years.append(year)
            #print(word)
        elif len(word) > 0 and word[-1] == '%' and word[0].isalpha() == False:
            #find the rating
            rating.append(word[:-1])#without %
            #print(word)
        

    #print(len(rating),len(link),len(review),len(date))
    if not (len(review) == len(link) == len(rating) == len(date) == len(months) == len(years) == len(album) == len(band) == len(author)  == len(albumlink)):
        print("ERROR year =",year,"month =",month)#check if no error, all should be equal due to constant no. of entries per page
        print(len(review), len(link), len(rating), len(date), len(months), len(years), len(album), len(band),len(author),  len(albumlink))
        return -1,N
    link_final += link #finding https
    review_final += review #Review titles only
    rating_final += rating #As a percentage
    date_final += date #month + date
    months_final += months
    years_final += years
    album_final += album
    albumlink_final += albumlink
    author_final += author
    band_final += band
    #texts_final += texts #actual review texts
    #sentiments_final += sentiments
    return len(rating),N # no error, return number of entries

In [4]:
def convert_to_csv():
    df = pd.DataFrame()
    df['User'] = author_final
    df['Date'] = date_final
    df['Year'] = years_final
    df['Month'] = months_final
    df['Release'] = album_final
    df['Band'] = band_final
    df['Score'] = rating_final
    df['Link'] = link_final
    df['Album_link'] = albumlink_final
    df['Review_title'] = review_final
    #df['Text'] = texts_final
    #df['Sentiment'] = sentiments_final
    df.to_csv('reviews_full_v1.csv', index = False)
    return df

scrape(2002,8,1)

In [5]:
for year in range(2002,2018):
    for month in range(1,13):
        phase = 0
        while scrape(year,month,phase)[0] == 200:
            #convert_to_csv()
            phase += 1
        if year == 2017 and month == 6:
            break
    print(year,len(rating_final),phase)
    if year == 2017 and month == 6:
        break
convert_to_csv()

2002 679 0
2003 2971 1
2004 7887 2
2005 11756 1
2006 16432 1
2007 24306 3
2008 33522 3
2009 42407 2
2010 49447 3
2011 57625 3
2012 65404 2
2013 71859 2
2014 78585 4
2015 84228 3
2016 90237 4
2017 93434 4


,User,Date,Year,Month,Release,Band,Score,Link,Album_link,Review_title
0,SpookyApparition,July 11 2002,2002,7,Chaosphere,Meshuggah,82,https://www.metal-archives.com/reviews/Meshugg...,https://www.metal-archives.com/albums/Meshugga...,Technical metal at it's finest.
1,SpookyApparition,July 14 2002,2002,7,Necroticism_-_Descanting_the_Insalubrious,Carcass,88,https://www.metal-archives.com/reviews/Carcass...,https://www.metal-archives.com/albums/Carcass/...,Carcass at their finest.
2,MarkRyan,July 15 2002,2002,7,Angels_Fall_First,Nightwish,90,https://www.metal-archives.com/reviews/Nightwi...,https://www.metal-archives.com/albums/Nightwis...,On par with their other stuff...
3,Fear,July 16 2002,2002,7,Dark_Genesis,Iced_Earth,87,https://www.metal-archives.com/reviews/Iced_Ea...,https://www.metal-archives.com/albums/Iced_Ear...,Brilliance in Repackaging
4,Esoteric,July 16 2002,2002,7,A_Night_at_the_Opera,Blind_Guardian,70,https://www.metal-archives.com/reviews/Blind_G...,https://www.metal-archives.com/albums/Blind_Gu...,Somewhat Disappointing
...,...,...,...,...,...,...,...,...,...,...
93429,6CORPSE6GRINDER6,June 30 2017,2017,6,Chemical_Assault,Violator,85,https://www.metal-archives.com/reviews/Violato...,https://www.metal-archives.com/albums/Violator...,"Take your life back, in the pit!"
93430,6CORPSE6GRINDER6,June 30 2017,2017,6,Path_of_Fire,Aeon,65,https://www.metal-archives.com/reviews/Aeon/Pa...,https://www.metal-archives.com/albums/Aeon/Pat...,As mediocre as it's predecessor
93431,6CORPSE6GRINDER6,June 30 2017,2017,6,Violent_Mosh,Violator,80,https://www.metal-archives.com/reviews/Violato...,https://www.metal-archives.com/albums/Violator...,Officers are dirtier than the criminals they c...
93432,Wacke,June 30 2017,2017,6,Resurrection,Chimaira,89,https://www.metal-archives.com/reviews/Chimair...,https://www.metal-archives.com/albums/Chimaira...,What doesn't kill us makes us stronger.
